In [1]:
# import package

# model
import copy
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch import optim
from torch.optim.lr_scheduler import StepLR

# dataset and transformation
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os

# display images
from torchvision import utils
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
class BasicBlock(nn.Module):
    def __init__(self, input_ch, output_ch, stride=1):
        super().__init__()

        self.basick_layer = nn.Sequential(
            nn.Conv2d(input_ch, output_ch, kernel_size=3,
                      stride=stride, padding=1),
            nn.BatchNorm2d(output_ch),
            nn.ReLU(),
            nn.Conv2d(output_ch, output_ch,
                      kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(output_ch),
        )

        self.shortcut = nn.Sequential()

        # if stride != 1:
        #     self.shortcut = nn.Sequential(
        #         nn.Conv2d(input_ch, output_ch, kernel_size=1, stride=2),
        #         nn.BatchNorm2d(output_ch)
        #     )

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.basick_layer(x)
        x = self.relu(x)
        return x

In [11]:
class Resnet34(nn.Module):
    def __init__(self, block, num_classes=2, init_weight=True):
        super().__init__()
        self.in_channel = 64

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.conv2_x = self.make_block(block, out_channel=64, num_block=3, stride=1)
        self.conv3_x = self.make_block(block, out_channel=128, num_block=4, stride=2)
        self.conv4_x = self.make_block(block, out_channel=256, num_block=6, stride=2)
        self.conv5_x = self.make_block(block, out_channel=512, num_block=3, stride=2)
        self.avgpool1 = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(512, num_classes)

        if init_weight:
            self.weights_initialize()

    def make_block(self, block, out_channel, num_block, stride):
        stride_list = [stride] + [1] * (num_block - 1)
        layer = []
        for i in stride_list:
            layer.append(block(self.in_channel, out_channel, i))
            self.in_channel = out_channel
        return nn.Sequential(*layer)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2_x(x)
        x = self.conv3_x(x)
        x = self.conv4_x(x)
        x = self.conv5_x(x)
        x = self.avgpool1(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

    def weights_initialize(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(
                    m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cuda


In [13]:
model = Resnet34(BasicBlock).to(device)
print(model)

Resnet34(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2_x): Sequential(
    (0): BasicBlock(
      (basick_layer): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): Sequential()
      (relu): ReLU()
    )
    (1): BasicBlock(
      (basick_layer): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.

In [14]:
summary(model, input_size=(3, 224, 224), device=device.type)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,928
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,928
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,928
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,